<a href="https://colab.research.google.com/github/Danish-Hussain-DSAI/facial-expression-recognition-project-/blob/main/facial_expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#unzip and store images in origin folder
!7za -y x "/content/drive/MyDrive/facial expression project/data/image/origin.7z.*"


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/facial expression project/data/image/                                                                      8 files, 8113576419 bytes (7738 MiB)
  0%    
Extracting archive: /content/drive/MyDrive/facial expression project/data/image/origin.7z.001
--
Path = /content/drive/MyDrive/facial expression project/data/image/origin.7z.001
Type = Split
Physical Size = 1048576000
Volumes = 8
Total Physical Size = 8113576419
----
Path = origin.7z
Size = 8113576419
--
Path = origin.7z
Type = 7z
Physical Size = 8113576419
Headers Size = 863607
Method = LZMA:25
Solid = +
Blocks = 2

  0%  

In [1]:
targets=r"/content/drive/MyDrive/facial expression project/data/label/label.lst"
input_images=r"/content/drive/MyDrive/facial expression project/origin"

In [2]:
#creating data frames
import pandas as pd
targets_table =pd.read_csv(targets, sep=" ",header=None)
table_cols="image_name face_id_in_image face_box_top face_box_left face_box_right face_box_bottom face_box_cofidence expression_label".split(" ")
targets_table.columns=table_cols
targets_table.head()

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0


In [3]:
#select images for training and testing
training_images=targets_table[targets_table.face_box_cofidence>60]
testing_images=targets_table[targets_table.face_box_cofidence<20]

In [4]:
print("training",training_images.shape)
print("testing",testing_images.shape)

training (33374, 8)
testing (12552, 8)


In [5]:
import os
import numpy as np
import cv2

In [ ]:
#load images for training
 train_set_x=[]
 train_set_y=[]
 for i, row in training_images.iterrows():
     img_name = row["image_name"]
     x1 = row["face_box_left"]
     x2 = row["face_box_right"]
     y1 = row["face_box_top"]
     y2 = row["face_box_bottom"]
     target=row["expression_label"]
     img_path =os.path.join(input_images, img_name)


     # Load the image
     filtered_image = cv2.imread(img_path)
     print(filtered_image)

     if filtered_image is not None:
        cropped_image=filtered_image[y1:y2,x1:x2]
     else:
        continue

     if cropped_image is not None:

       cropped_resized=cv2.resize(cropped_image,(64, 64))

     else:
      continue

     train_set_x.append(cropped_resized)
     train_set_y.append(target)












Streaming output truncated to the last 5000 lines.
  [ 34  32  32]
  [ 50  46  45]]]
[[[154  81   0]
  [154  81   0]
  [152  82   0]
  ...
  [150  81   1]
  [150  81   1]
  [150  81   1]]

 [[154  81   0]
  [154  81   0]
  [152  82   0]
  ...
  [150  81   1]
  [150  81   1]
  [150  81   1]]

 [[154  81   0]
  [154  81   0]
  [152  82   0]
  ...
  [150  81   1]
  [150  81   1]
  [150  81   1]]

 ...

 [[140 140 140]
  [140 140 140]
  [140 140 140]
  ...
  [140 140 140]
  [140 140 140]
  [140 140 140]]

 [[140 140 140]
  [140 140 140]
  [140 140 140]
  ...
  [140 140 140]
  [140 140 140]
  [140 140 140]]

 [[140 140 140]
  [140 140 140]
  [140 140 140]
  ...
  [140 140 140]
  [140 140 140]
  [140 140 140]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 

In [ ]:
print(len(train_images_x))
print(len(train_images_y))

In [ ]:
#load images for testing
 test_set_x=[]
 test_set_y=[]
 j=0

 for i, row in testing_images.iterrows():
     img_name = row["image_name"]
     x1 = row["face_box_left"]
     x2 = row["face_box_right"]
     y1 = row["face_box_top"]
     y2 = row["face_box_bottom"]
     target=row["expression_label"]
     img_path =os.path.join(input_images, img_name)

     j+=1
     # Load the image
     filtered_image = cv2.imread(img_path)
     print(j)



     if filtered_image is not None:
        cropped_image=filtered_image[y1:y2,x1:x2]
     else:
        continue

     if cropped_image is not None:

       cropped_resized=cv2.resize(cropped_image,(64, 64))

     else:
      continue

     test_set_x.append(cropped_resized)
     test_set_y.append(target)

In [ ]:
print(len(test_set_x))
print(len(test_set_y))

70
70


In [ ]:
#convert into numpy array
import numpy as np
train_x=np.array(train_set_x)    #training images
train_y=np.array(train_set_y)    #training labels
test_x=np.array(test_set_x)      #test images
test_y=np.array(test_set_y)      #test labels




In [ ]:
print(len(train_x))

86


In [ ]:
#normalization
train_x_normalized = train_x / 255.0
test_x_normalized = test_x / 255.0

train_x = train_x_normalized
test_x = test_x_normalized

In [ ]:
#encoding
from keras.utils import to_categorical
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)


In [ ]:
#sampling
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
x_resampled, y_resampled = ros.fit_resample(train_x.reshape(-1, 64*64*3), train_y)

train_x=x_resampled.reshape(-1,64*64*3)
train_y=y_resampled


In [ ]:
#encoding
from keras.utils import to_categorical

num_classes = len(np.unique(train_y))
train_y_encoded = to_categorical(train_y, num_classes)


test_y_encoded = to_categorical(test_y, num_classes)

train_y=train_y_encoded
test_y=test_y_encoded


In [ ]:
#model training and testing
import numpy as np
from keras.applications import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.utils import to_categorical
from skimage.transform import resize



input_shape = (224, 224, 3)
train_x_resized = np.array([resize(image, input_shape) for image in train_x])
test_x_resized = np.array([resize(image, input_shape) for image in test_x])


base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)


for layer in base_model.layers:
    layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


train_y_encoded = to_categorical(train_y, num_classes)
test_y_encoded = to_categorical(test_y, num_classes)


batch_size = 32
epochs = 10

model.fit(train_x_resized, train_y_encoded, batch_size=batch_size, epochs=epochs, validation_data=(test_x_resized, test_y_encoded))

loss, accuracy = model.evaluate(test_x_resized, test_y_encoded)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')
